# Climatic attributes

Notebook to create the file `CAMELS_DE_climatic_attributes.csv`.  

columns in CAMELS-GB:
- gauge_id / camels_id ?
- p_mean [mm/day]
- pet_mean [mm/day]
- aridity [-]
- p_seasonality [-]
- frac_snow [-]
- high_prec_freq [days/yr]
- high_prec_dur [days]
- high_prec_timing [season]
- low_prec_freq [days/yr]
- low_prec_dur [days]
- low_prec_timing [season]


In [36]:
from tqdm import tqdm
from glob import glob
import pandas as pd

In [46]:
# get camels_ids from hydromet timeseries
camels_ids = [camels_id.split("_")[-1].split(".csv")[0] for camels_id in glob("../output_data/camels_de/timeseries/*.csv")]

# sort camels_ids
camels_ids = sorted(camels_ids)

print(f"Total number of stations in CAMELS-DE v1: {len(camels_ids)}")

Total number of stations in CAMELS-DE v1: 1460


In [47]:
# dataframe to store results
df_results = pd.DataFrame()

## p_mean

*mean daily preciptiation*

In [49]:
for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # calculate p_mean from precipitation_mean
    p_mean = df["precipitation_mean"].mean()

    # add to results
    df_results.loc[camels_id, "p_mean"] = round(p_mean, 2)

df_results.head()

100%|██████████| 1460/1460 [01:41<00:00, 14.39it/s]


,p_mean
DE110000,2.96
DE110010,2.91
DE110020,2.54
DE110030,2.45
DE110040,2.61


## pet_mean

*mean daily PET*

At the moment, we do not include PET data directly in CAMELS-DE.

## aridity

*aridity, calculated as the ratio of mean daily potential evapotranspiration to mean daily precipitation*

No PET in CAMELS-DE.

## p_seasonality

*seasonality and timing of precipitation (estimated using sine curves to represent the annual temperature and precipitation cycles; positive (negative) values indicate that precipitation peaks in summer (winter) and values close to zero indicate uniform precipitation throughout the year)*

TODO: Ralf - how to calculate this?

from: https://github.com/camels-ch/camels/blob/c584d823b0fc3f4b0e9ea21b638029bd107114a5/hydro_climate_attributes/climate_indices.R#L65-L82

In [113]:
import numpy as np
from scipy.optimize import curve_fit

# Define the sine function to fit
def sine_curve(day_of_year, mean_value, amplitude, phase_shift):
    return mean_value * (1 + amplitude * np.sin(2 * np.pi * (day_of_year - phase_shift) / 365.25))

for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # convert date to datetime
    df["date"] = pd.to_datetime(df["date"])

    # Create a time variable that represents the day of the year
    df["day_of_year"] = df["date"].dt.dayofyear

    # Get the mean precipitation and temperature
    average_precipitation = df["precipitation_mean"].mean()
    average_temperature = df["temperature_mean"].mean()

    # Get the first guess for the phase shift
    initial_phase_shift_guess_prec = 90 - df["precipitation_mean"].idxmax() * 30
    initial_phase_shift_guess_prec = initial_phase_shift_guess_prec % 360

    initial_phase_shift_guess_temp = -90

    # Fit a sine curve to the precipitation and temperature data
    optimized_parameters_prec, parameter_covariances_prec = curve_fit(sine_curve, df["day_of_year"], df["precipitation_mean"], p0=[average_precipitation, 0.4, initial_phase_shift_guess_prec])
    optimized_parameters_temp, parameter_covariances_temp = curve_fit(sine_curve, df["day_of_year"], df["temperature_mean"], p0=[average_temperature, 5, initial_phase_shift_guess_temp])

    # The phase shifts are optimized_parameters[2]
    precipitation_seasonality = optimized_parameters_prec[2]
    temperature_seasonality = optimized_parameters_temp[2]

    # The amplitudes are optimized_parameters[1]
    amplitude_prec = optimized_parameters_prec[1]
    amplitude_temp = optimized_parameters_temp[1]

    # Calculate p_seasonality
    p_seasonality = amplitude_prec * np.sign(amplitude_temp) * np.cos(2 * np.pi * (precipitation_seasonality - temperature_seasonality) / 365.25)

    # Add to results
    df_results.loc[camels_id, "p_seasonality"] = round(p_seasonality, 2)

100%|██████████| 1460/1460 [02:19<00:00, 10.43it/s]


## frac_snow

*fraction of precipitation falling as snow (for days colder than 0°C)*

TODO Ralf: nehmen wir hier Tage an denen temperature_mean < 0°C ist? Oder temperature_min < 0°C? Oder temperature_max < 0°C?

In [122]:
for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # fraction of precipitation falling as snow (for days colder than 0°C)
    sum_precip_snow = df[df["temperature_mean"] < 0]["precipitation_mean"].sum()
    sum_precip_water = df[df["temperature_mean"] >= 0]["precipitation_mean"].sum()
    frac_snow = sum_precip_snow / (sum_precip_snow + sum_precip_water)

    # add to results
    df_results.loc[camels_id, "frac_snow"] = round(frac_snow, 2)

df_results.head()

100%|██████████| 1460/1460 [00:56<00:00, 25.91it/s]


,p_mean,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,low_prec_timing,p_seasonality
DE110000,2.96,0.12,15.77,1.19,djf,212.04,3.70,son,0.02
DE110010,2.91,0.12,15.90,1.19,djf,212.50,3.70,son,0.03
DE110020,2.54,0.10,15.14,1.18,jja,212.97,3.72,son,0.20
DE110030,2.45,0.09,15.13,1.17,jja,214.01,3.73,son,0.25
DE110040,2.61,0.07,17.16,1.18,jja,224.17,3.74,son,0.40


## high_prec_freq

*frequency of high precipitation days (≥ 5 times mean daily precipitation)* [days/yr]

In [76]:
for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # mean precipitation
    p_mean = df["precipitation_mean"].mean()

    # number of days >= 5 times mean precipitation
    n_days_high_freq = len(df[df["precipitation_mean"] >= 5 * p_mean]) / 70 # 70 years of data

    # add to results
    df_results.loc[camels_id, "high_prec_freq"] = round(n_days_high_freq, 2)

df_results.head()

  0%|          | 0/1460 [00:00<?, ?it/s]

100%|██████████| 1460/1460 [01:34<00:00, 15.44it/s]


,p_mean,frac_snow,high_prec_freq
DE110000,2.96,0.117822,15.77
DE110010,2.91,0.115780,15.90
DE110020,2.54,0.102980,15.14
DE110030,2.45,0.094166,15.13
DE110040,2.61,0.072553,17.16


## high_prec_dur

*average duration of high precipitation events (number of consecutive days ≥ 5 times mean daily precipitation)* [days]

In [81]:
# initialize variables to keep track of high precipitation event
high_precip_streaks = []
current_streak = 0

for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # mean precipitation
    p_mean = df["precipitation_mean"].mean()

    # iterate over the DataFrame's rows
    for precip in df["precipitation_mean"]:
        if precip >= 5 * p_mean:
            # if the day's precipitation is higher 5 times mean precipitation, increment the current streak
            current_streak += 1
        elif current_streak > 0:
            # if the day's precipitation is not high and there's a current streak, add it to the list of all streaks and reset it
            high_precip_streaks.append(current_streak)
            current_streak = 0

    # if there's a current streak at the end of the DataFrame, add it to the list of all streaks
    if current_streak > 0:
        high_precip_streaks.append(current_streak)

    # calculate the average streak length for the station
    average_streak_length = sum(high_precip_streaks) / len(high_precip_streaks) if high_precip_streaks else 0

    # add to results
    df_results.loc[camels_id, "high_prec_dur"] = round(average_streak_length, 2)

df_results.head()

100%|██████████| 1460/1460 [01:52<00:00, 12.98it/s]


,p_mean,frac_snow,high_prec_freq,high_prec_dur
DE110000,2.96,0.117822,15.77,1.19
DE110010,2.91,0.115780,15.90,1.19
DE110020,2.54,0.102980,15.14,1.18
DE110030,2.45,0.094166,15.13,1.17
DE110040,2.61,0.072553,17.16,1.18


## high_prec_timing

*season during which most high precipitation days (≥ 5 times mean daily precipitation) occur (djf December, January, February; mam March, April, May; jja June, July, August; son September, October, November). If two seasons register the same number of events, a value of NaN is given.* [season]

In [99]:
# Define a function to get the season from a date
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'djf'
    elif month in [3, 4, 5]:
        return 'mam'
    elif month in [6, 7, 8]:
        return 'jja'
    else:
        return 'son'

for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # parse date column
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

    # mean precipitation
    p_mean = df["precipitation_mean"].mean()

    # Initialize a dictionary to keep track of the number of high-precipitation days in each season
    season_counts = {'djf': 0, 'mam': 0, 'jja': 0, 'son': 0}

    # Iterate over the DataFrame's rows
    for index, row in df.iterrows():
        if row['precipitation_mean'] >= 5 * p_mean:
            # If the day's precipitation is high, increment the count for its season
            season_counts[get_season(row['date'])] += 1

    # Find the season with the most high-precipitation days
    max_count = max(season_counts.values())
    max_seasons = [season for season, count in season_counts.items() if count == max_count]

    # If there's a tie, return NaN
    if len(max_seasons) > 1:
        max_season = float('nan')
    else:
        max_season = max_seasons[0]

    # Add to results
    df_results.loc[camels_id, "high_prec_timing"] = max_season

df_results.head()

100%|██████████| 1460/1460 [15:32<00:00,  1.57it/s]


## low_prec_freq

*frequency of low precipitation days (< 1 mm/day)* [days/yr]

In [101]:
for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # number of days < 1 mm of precipitation
    n_days_low_freq = len(df[df["precipitation_mean"] < 1]) / 70 # 70 years of data

    # add to results
    df_results.loc[camels_id, "low_prec_freq"] = round(n_days_low_freq, 2)

df_results.head()

100%|██████████| 1460/1460 [01:44<00:00, 13.93it/s]


,p_mean,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq
DE110000,2.96,0.117822,15.77,1.19,djf,212.04
DE110010,2.91,0.115780,15.90,1.19,djf,212.50
DE110020,2.54,0.102980,15.14,1.18,jja,212.97
DE110030,2.45,0.094166,15.13,1.17,jja,214.01
DE110040,2.61,0.072553,17.16,1.18,jja,224.17


## low_prec_dur

*average duration of dry periods (number of consecutive days < 1 mm/day)* [days]

In [102]:
# initialize variables to keep track of high precipitation event
low_precip_streaks = []
current_streak = 0

for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # iterate over the DataFrame's rows
    for precip in df["precipitation_mean"]:
        if precip < 1:
            # if the day's precipitation is higher 5 times mean precipitation, increment the current streak
            current_streak += 1
        elif current_streak > 0:
            # if the day's precipitation is not high and there's a current streak, add it to the list of all streaks and reset it
            low_precip_streaks.append(current_streak)
            current_streak = 0

    # if there's a current streak at the end of the DataFrame, add it to the list of all streaks
    if current_streak > 0:
        low_precip_streaks.append(current_streak)

    # calculate the average streak length for the station
    average_streak_length = sum(low_precip_streaks) / len(low_precip_streaks) if low_precip_streaks else 0

    # add to results
    df_results.loc[camels_id, "low_prec_dur"] = round(average_streak_length, 2)

df_results.head()

100%|██████████| 1460/1460 [01:56<00:00, 12.52it/s]


,p_mean,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur
DE110000,2.96,0.117822,15.77,1.19,djf,212.04,3.70
DE110010,2.91,0.115780,15.90,1.19,djf,212.50,3.70
DE110020,2.54,0.102980,15.14,1.18,jja,212.97,3.72
DE110030,2.45,0.094166,15.13,1.17,jja,214.01,3.73
DE110040,2.61,0.072553,17.16,1.18,jja,224.17,3.74


## low_prec_timing

*season during which most dry days (< 1mm day-1) occur (djf December, January, February; mam March, April, May; jja June, July, August; son September, October, November). If two seasons register the same number of events, a value of NaN is given.* [season]

In [105]:
# Define a function to get the season from a date
def get_season(date):
    month = date.month
    if month in [12, 1, 2]:
        return 'djf'
    elif month in [3, 4, 5]:
        return 'mam'
    elif month in [6, 7, 8]:
        return 'jja'
    else:
        return 'son'

for camels_id in tqdm(camels_ids):
    # read camels de hydromet timeseries data
    df = pd.read_csv(f"../output_data/camels_de/timeseries/CAMELS_DE_hydromet_timeseries_{camels_id}.csv")

    # parse date column
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")

    # Initialize a dictionary to keep track of the number of low-precipitation days in each season
    season_counts = {'djf': 0, 'mam': 0, 'jja': 0, 'son': 0}

    # Iterate over the DataFrame's rows
    for index, row in df.iterrows():
        if row['precipitation_mean'] < 1:
            # If the day's precipitation is low, increment the count for its season
            season_counts[get_season(row['date'])] += 1

    # Find the season with the most low-precipitation days
    max_count = max(season_counts.values())
    max_seasons = [season for season, count in season_counts.items() if count == max_count]

    # If there's a tie, return NaN
    if len(max_seasons) > 1:
        max_season = float('nan')
    else:
        max_season = max_seasons[0]

    # Add to results
    df_results.loc[camels_id, "low_prec_timing"] = max_season

df_results.head()

100%|██████████| 1460/1460 [16:37<00:00,  1.46it/s]


,p_mean,frac_snow,high_prec_freq,high_prec_dur,high_prec_timing,low_prec_freq,low_prec_dur,low_prec_timing
DE110000,2.96,0.117822,15.77,1.19,djf,212.04,3.70,son
DE110010,2.91,0.115780,15.90,1.19,djf,212.50,3.70,son
DE110020,2.54,0.102980,15.14,1.18,jja,212.97,3.72,son
DE110030,2.45,0.094166,15.13,1.17,jja,214.01,3.73,son
DE110040,2.61,0.072553,17.16,1.18,jja,224.17,3.74,son


## Save results

Create file `CAMELS_DE_climatic_attributes.csv`.

In [123]:
# save results
df_results.to_csv(f"../output_data/camels_de/CAMELS_DE_climatic_attributes.csv", index_label="camels_id")